<a href="https://colab.research.google.com/github/kidujm/data_mini_project/blob/main/%EC%9C%A0%EC%A0%95%EB%AF%BC_0211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 정제과정
- 기준 컬럼 이름 맞추기 (행정동_코드)
- 경기도 행정동 코드 8자리로 맞추기
- 행정동 코드 데이터타입 맞추기
- 서울(분기)/경기는(월 기준) 데이터 맞추기
- 2025년 1분기 = 2025 01~03
- 경기도의 시간대 코드 유동인구수 합치기


In [47]:
import pandas as pd


In [48]:
file_g = "경기도_유동인구_한글컬럼.csv"
file_s = "서울시 상권분석서비스(길단위인구-행정동).csv"
file_m = "꿀빵떡1_최종.csv"

df_g = pd.read_csv(file_g, encoding="utf-8-sig")   # 경기도
df_s = pd.read_csv(file_s, encoding="cp949")       # 서울시
df_m = pd.read_csv(file_m, encoding="utf-8-sig")   # 꿀빵떡(매장)


In [52]:
# 행정동 코드 컬럼명 맞추기
df_m = df_m.rename(columns={"행정동코드": "행정동_코드"})

In [50]:
# 경기도 행정동 코드 > 8 자리
df_g["행정동_코드"] = (
    df_g["행정동_코드"]
    .astype(str)
    .str.extract(r"(\d+)")[0]   # 숫자만 뽑기
    .str[:8]                    # 앞 8자리
)

In [53]:
df_g["행정동_코드"] = pd.to_numeric(df_g["행정동_코드"], errors="coerce").astype("Int64")
df_s["행정동_코드"] = pd.to_numeric(df_s["행정동_코드"], errors="coerce").astype("Int64")
df_m["행정동_코드"] = pd.to_numeric(df_m["행정동_코드"], errors="coerce").astype("Int64")

print(df_g["행정동_코드"].dtype, df_s["행정동_코드"].dtype, df_m["행정동_코드"].dtype)


Int64 Int64 Int64


In [54]:
print("서울 분기값 예시:", df_s["기준_년분기_코드"].dropna().unique()[:10])
print("경기 월값 예시:", df_g["기준_연월"].dropna().unique()[:10])


서울 분기값 예시: [20253 20252 20251 20244 20243 20242 20241 20234 20233 20232]
경기 월값 예시: [202503 202209 201902 201811 202406 202403 201910 201807 202207 201911]


In [55]:
# 분기를 기준으로 데이터로 만듬
# 시간대별로 쪼개진 데이터를 합침

# 2025년 1분기(1~3월)
df_g_q1 = df_g[df_g["기준_연월"].between(202501, 202503)].copy()

# TZ07~TZ10
target_tz = ["TZ07", "TZ08", "TZ09", "TZ10"]
df_g_q1 = df_g_q1[df_g_q1["시간대_코드"].isin(target_tz)].copy()

# 행정동별 합계
df_pop_g = (
    df_g_q1.groupby("행정동_코드", as_index=False)["유동인구_수"]
    .sum()
    .rename(columns={"유동인구_수": "총유동인구_수"})
)

display(df_pop_g.head())
print("경기 행정동 개수:", df_pop_g["행정동_코드"].nunique())


,행정동_코드,총유동인구_수
0,41111560,10695132.56
1,41111566,10695132.56
2,41111571,10695132.56
3,41111572,10695132.56
4,41111573,10695132.56


경기 행정동 개수: 572


In [56]:
df_s_q1 = df_s[df_s["기준_년분기_코드"] == 20251].copy()

df_pop_s = df_s_q1[["행정동_코드", "총_유동인구_수"]].copy()
df_pop_s = df_pop_s.rename(columns={"총_유동인구_수": "총유동인구_수"})

# 혹시 중복 행정동이 있으면 합치기(안전)
df_pop_s = df_pop_s.groupby("행정동_코드", as_index=False)["총유동인구_수"].sum()

display(df_pop_s.head())
print("서울 행정동 개수:", df_pop_s["행정동_코드"].nunique())


,행정동_코드,총유동인구_수
0,11110515,3322244
1,11110530,3978820
2,11110540,948998
3,11110550,902967
4,11110560,901766


서울 행정동 개수: 425


In [57]:
df_pop = pd.concat([df_pop_g, df_pop_s], ignore_index=True)
df_pop = df_pop.groupby("행정동_코드", as_index=False)["총유동인구_수"].sum()

display(df_pop.head())
print("전체 행정동 개수:", df_pop["행정동_코드"].nunique())


,행정동_코드,총유동인구_수
0,11110515,3322244.0
1,11110530,3978820.0
2,11110540,948998.0
3,11110550,902967.0
4,11110560,901766.0


전체 행정동 개수: 997


In [58]:
df_result = df_m.merge(df_pop, on="행정동_코드", how="left")

match_rate = df_result["총유동인구_수"].notna().mean()
print("매칭률:", round(match_rate*100, 2), "%")

df_result.to_csv("꿀빵떡1_총유동인구추가_2025Q1.csv", index=False, encoding="utf-8-sig")
print("저장 완료: 꿀빵떡1_총유동인구추가_2025Q1.csv")

display(df_result.head())


매칭률: 88.08 %
저장 완료: 꿀빵떡1_총유동인구추가_2025Q1.csv


,상가업소번호,brand,상호명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,월간 검색량,블로그누적발행,방문자사진수,역명,역_거리,역_거리(max1500),추정_개업일,방문자리뷰수,블로그리뷰수,총유동인구_수
0,MA010120220803665593,투다리,투다리,I2,음식,I211,주점,I21104,요리 주점,I56219,...,20.0,45.00,36.0,삼각지역,15.0,15.0,2022년 8월 18일 목요일,191.0,11.0,5623799.0
1,MA010120220803650891,투다리,투다리,I2,음식,I211,주점,I21104,요리 주점,I56219,...,20.0,97.00,11.0,외대앞역,218.0,218.0,2016년 12월 23일 금요일,113.0,10.0,8871536.0
2,MA010120220804077811,투다리,투다리,I2,음식,I211,주점,I21104,요리 주점,I56219,...,70.0,261.03,46.0,자양역,974.0,974.0,2017년 9월 9일 토요일,239.0,20.0,5185483.0
3,MA010120220808007750,역전할머니,역전할머니맥주강서,I2,음식,I211,주점,I21103,생맥주 전문,I56219,...,20.0,231.00,79.0,가양역,985.5,985.5,2019년 4월 22일 월요일,612.0,60.0,5457855.0
4,MA010120220809830428,투다리,투다리,I2,음식,I211,주점,I21104,요리 주점,I56219,...,20.0,72.00,45.0,종합운동장역,409.0,409.0,2022년 10월 3일 월요일,71.0,16.0,8029149.0


In [59]:
# 병합 결과가 df_result 라고 가정
unmatched = df_result[df_result["총유동인구_수"].isna()].copy()

# 매칭 실패 코드(중복 제거)
unmatched_codes = unmatched["행정동_코드"].dropna().drop_duplicates().sort_values()

print("매칭 실패 행 수:", len(unmatched))
print("매칭 실패 코드 개수:", unmatched_codes.size)

display(unmatched_codes.head(30))


매칭 실패 행 수: 87
매칭 실패 코드 개수: 47


,행정동_코드
12,11740525
610,41117585
267,41117593
469,41150578
710,41150595
273,41150630
583,41173552
675,41173559
663,41210655
685,41220635


In [60]:
# 파일명 지정
output_file = "꿀빵떡1_총유동인구추가_2025Q1.csv"

# CSV 저장 (한글 깨짐 방지 위해 utf-8-sig)
df_result.to_csv(output_file, index=False, encoding="utf-8-sig")

print("저장 완료:", output_file)


저장 완료: 꿀빵떡1_총유동인구추가_2025Q1.csv


### 꿀빵떡1_총유동인구추가_2025Q1.csv
- 총 유동인구수 추가했는데 결측값 생김 이부분 확인 후 수정 필요함